# Рекомендательная система на основе Surprise

Набор данных - Goodreads books 10k, оригинальный, хранится по адресу расположенный по адресу: https://github.com/zygmuntz/goodbooks-10k    

Пакет для построения рекомендательной системы - Surprise



In [2]:
import pandas as pd, numpy as np


In [3]:
ratings = pd.read_csv('./goodbooks-10k-master/ratings.csv') #поставленные оценки
print('Ratings:',ratings.shape[0])
ratings.head()

Ratings: 5976479


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [4]:
mn = ratings.rating.min()
mx = ratings.rating.max()
print(f'Min rating {mn}, max rating {mx}')

Min rating 1, max rating 5


Преобразовываем данные во внутренний формат и делим на обучающий и тестовый фрагменты. 

In [5]:
from surprise.model_selection import train_test_split
from surprise import Dataset,  Reader

# Преобразовываю данные в формат dataset модуля surprise 
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], Reader(rating_scale=(1,5)))
# разбиваю на обучающую и тестовую части
trainset, testset = train_test_split(data, test_size=0.2, random_state=13)

В пакет surprise не входят методы оценки precision@k и recall@k. Зададим функцию для расчета. Необходимо отметить, что наличие численного рейтинга позволяет разделить оценки на две группы - понравившиеся и не понраившиеся. Я предполанаю по умолчанию что книги с проставленным рейтингом выше 3.5 понравились, а ниже - нет. 

In [6]:
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items 
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of relevant items in top k 
        #n_rel_k = sum((true_r >= threshold) for (_, true_r) in user_ratings[:k])


        # Number of recommended items in top k EQUALS TO K
        #n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended relevan items in top k. 
        # precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        precisions[uid] = n_rel_and_rec_k / k if k !=0 else 0

        # Recall@K: Proportion of relevant items in top k that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


Посмотрим на качество модели с параметрами из коробки:

In [9]:
from surprise import SVD, accuracy

model = SVD(verbose=True)
model.fit(trainset)
svd_prediction = model.test(testset)
print(accuracy.rmse(svd_prediction))


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.8292
0.8292215296685491


In [10]:

precisions_10, recall_10 = precision_recall_at_k(svd_prediction, k=10)
print('Precision @10:', np.mean(list(precisions_10.values())))
print('Recall @10:', np.mean(list(recall_10.values())))

#print('Precision @10:', np.mean([ v for v in precisions_10.values()]))
#print('Recall @10:', np.mean([ v for v in recall_10.values()]))


Precision @10: 0.7605140011979635
Recall @10: 0.5245201679370142


## Резюме 

5 млн записей предъявили некоторую нагрузку для обработки (примерно 60 секунд на эпоху обучения).  За время обучения на порядок меньше, чем в случае LightFM, получено намного более высокое качество - в среднем 8 книги из 10 соответствуют ожиданиям пользователей (precision@10 = 0.7605). 

